# Check Guild

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import engine.models as GM

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Properties

In [8]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [9]:
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)
PD.DataFrame([guildMd.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,LastPlayed,CreateDate,Selected,ThroneLevel,CampaignWorld,TotalPower,MaxThieves,StorageGold,StorageGems,StorageWood,StorageStone,StorageIron,VaultGold,VaultGems,VaultWood,VaultStone,VaultIron,DungeonCheck
0,1,433,2024-08-03,2024-08-03,True,1,1,465,6,3320,0,890,0,0,661,209,890,0,0,2024-08-18


In [10]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(thieves).drop(['id',
        'Name', 'BasePower', 'Experience', 
        'Position', 'Wounds', 'Cooldown',
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
        'Attack', 'Damage', 'Defense',
        'Traverse', 'Sabotage', 'Perceive', 
        ], axis=1, errors='ignore')

,GuildFK_id,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,Health,Status,CooldownExpire
0,22,Burglar,1,1,153,6,0,0,0,58,Ready,None
1,22,Scoundrel,1,1,96,0,5,0,0,58,Ready,None
2,22,Ruffian,1,1,144,0,0,5,0,58,Ready,None
3,22,Burglar,1,1,72,4,0,0,0,58,Ready,None


In [11]:
thiefMds = GM.ThiefInGuild.objects.filter(GuildFK=guildMd)
thiefLs = []

for md in thiefMds:

    thiefDx = md.__dict__
    thiefLs.append(thiefDx)
    
    thiefItems = GM.ItemInGuild.objects.filter(ThiefFK=md).values()
    thiefDx['ItemCount'] = len(thiefItems)
    
    slots = ['weapon', 'armor', 'head', 'hands', 'feet', ]
    
    for sl in slots:
        itemCheck = [x for x in thiefItems if x['Slot']==sl]
        
        if itemCheck:
            item = itemCheck[0]
            res = item.pop('GuildFK_id')
            res = item.pop('ThiefFK_id')
            if sl in ['weapon', 'armor']: stat = item['Trait'][:3]
            else:     stat = 'skl' if item['Skill'] else 'cmb'
            item['icon'] = f"{sl}-{stat}.png"
        else:
            item = {}
            item['id'] = -1
            item['icon'] = f"{sl}-empty.png"

        thiefDx[sl] = item


PD.DataFrame(thiefItems)

,id,Slot,Name,Level,TotalLv,Power,Requirement,Trait,Skill,Combat,Magic,icon
0,312,weapon,Twin Daggers,1,1,24,Burglar,agi 1,None,dmg 1,None,weapon-agi.png


In [12]:
thiefDf = PD.DataFrame(thiefLs)
thiefDf = thiefDf.drop(['_state', 'GuildFK_id', 'BasePower', 'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
                        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd',
                        'Experience', 'Position', 'Wounds', 'Cooldown'], 
                    axis=1, errors='ignore')
thiefDf = thiefDf.sort_values('Power', ascending=False)
thiefDf

,id,Name,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,...,Perceive,Traverse,Status,CooldownExpire,ItemCount,weapon,armor,head,hands,feet
0,106,Anike,Burglar,1,1,153,6,0,0,0,...,5,5,Ready,None,5,"{'id': 299, 'Slot': 'weapon', 'Name': 'Twin Da...","{'id': 300, 'Slot': 'armor', 'Name': 'Brigandi...","{'id': 305, 'Slot': 'head', 'Name': 'Cowl', 'L...","{'id': 307, 'Slot': 'hands', 'Name': 'Gloves',...","{'id': 311, 'Slot': 'feet', 'Name': 'Boots', '..."
2,108,Steppenwulf,Ruffian,1,1,144,0,0,5,0,...,0,0,Ready,None,5,"{'id': 303, 'Slot': 'weapon', 'Name': 'Sword &...","{'id': 304, 'Slot': 'armor', 'Name': 'Chain Ma...","{'id': 306, 'Slot': 'head', 'Name': 'Helmet', ...","{'id': 308, 'Slot': 'hands', 'Name': 'Gauntlet...","{'id': 310, 'Slot': 'feet', 'Name': 'Greaves',..."
1,107,Weyland,Scoundrel,1,1,96,0,5,0,0,...,0,0,Ready,None,2,"{'id': 301, 'Slot': 'weapon', 'Name': 'Blackth...","{'id': 302, 'Slot': 'armor', 'Name': 'Vestment...","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
3,109,Adjani,Burglar,1,1,72,4,0,0,0,...,0,0,Ready,None,1,"{'id': 312, 'Slot': 'weapon', 'Name': 'Twin Da...","{'id': -1, 'icon': 'armor-empty.png'}","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"


In [13]:
guildItems = GM.ItemInGuild.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(guildItems).drop(['id'], axis=1, errors='ignore').sort_values('Slot')

,GuildFK_id,ThiefFK_id,Slot,Name,Level,TotalLv,Power,Requirement,Trait,Skill,Combat,Magic
1,22,106.0,armor,Brigandine,1,1,24,Burglar,agi 1,None,def 3,None
3,22,107.0,armor,Vestment,1,1,24,Scoundrel,cun 1,None,def 2,None
5,22,108.0,armor,Chain Maille,1,1,24,Ruffian,mig 1,None,def 1,None
10,22,NaN,feet,Boots,1,1,16,None,None,tra 5,None,None
11,22,108.0,feet,Greaves,1,1,16,None,None,None,att 1,None
12,22,106.0,feet,Boots,1,2,25,None,None,tra 5,None,agi 1
8,22,106.0,hands,Gloves,1,1,16,None,None,sab 5,None,None
9,22,108.0,hands,Gauntlet,1,1,16,None,None,None,dmg 1,None
6,22,106.0,head,Cowl,1,1,16,None,None,per 5,None,None
7,22,108.0,head,Helmet,1,1,16,None,None,None,def 1,None


### Stages

In [14]:
towers = GM.GuildTower.objects.all().values()
PD.DataFrame(towers).drop(['id'], axis=1, errors='ignore')

AttributeError: module 'engine.models' has no attribute 'GuildTower'

### Stores

In [15]:
unlocks = GM.ThiefUnlocked.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(unlocks).drop(['id'], axis=1, errors='ignore')

,GuildFK_id,ThiefFK_id
0,22,161
1,22,160


In [16]:
unlocks = GM.ItemUnlocked.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(unlocks).drop(['id'], axis=1, errors='ignore')

,GuildFK_id,ItemFK_id
0,22,1174
1,22,1167
2,22,1173
3,22,1169
